<h1>Stair landing

<h2>[Reference project model](http://stairs4u.com/code/stair_landing_minimum_widths.htm)</h2>
![alt text](model_reference.png)

<h4>Variables:</h4>  <b>alzata</b>: ideal riser, 
            <b>gradini</b>: stairs's number (maybe decimal number), 
            <b>resto</b>: eventual decimal of 'gradini', 
            <b>totGradiniIntero</b>: integer part of 'gradin' (the effettive stairs's number), 
            <b>newAlzata</b>: effettive riser, 
            <b>pedata</b>: ideal tread, 
            <b>yLanding</b>: landing's depth, 
            <b>pedataFraz</b>: sum treads stairs without landing's depth, 
            <b>newPedata</b>: effettive tread of each stair, 
            <b>larghezzaStair</b>: stair's width.

<h4>Description of the geometric method i'm going to implement:</h4> this feature allows you to create stairs and landings from third formal parameters passed to it. With the parameters passed, i will define the numbers of the steps with their tread and riser and the size of the landings.
The numbers of the steps according to the formal parameters can be decimal. To resolve this issue before I go on to take the integer part of the result of the division, and the rest of the division is proportionally added to each stair.
To create the stairs and landing of the first flight use a for that step will create the single stair according to the formal parameters, through the use of CUBOID. This starts at 1 for up to the total number of steps previously obtained.
For the second ramp simply I do an appropriate rotation of 180 ° and translation and assemble it with the first ramp.
The translation will be made on all 3 axes, the x-axis for the accuracy of a factor dx; the y-axis by a factor equal to the product between the tread and the number of steps of the single ramp, net of a tread; on the Z axis by a factor equal to the product between the riser and the number of the single ramp steps.
The ramps are assembled with their diagonal that goes to fill the space at the bottom of every single step.
This diagonal part as a list that gradually will add 3 points according to past measures as formal parameters, after which it exploits MKPOL to draw lines on the 3 axis, and PROD to make it three-dimensional, and with proper translation and rotation it fill the stairs (after being STRUCT).

Import libraries

In [28]:
from pyplasm import *
from larlib import *

Define principal function with 3 formal parameters, named dx,dy,dz

In [29]:
"""input: dx,dy,dz
   output: Hpc Object - Stairs and landing"""

#2alzata+pedata=64cm ideal formule of stair's size
def ggpl_stair_landing(dx,dy,dz):
    
    #variables
    alzata = .2
    gradini = dz/alzata    
    resto = modf(gradini)[0]    
    totGradiniIntero = modf(gradini)[1]
    newAlzata = dz/totGradiniIntero
    pedata =.64-(2*newAlzata)
    yLanding = dy/4.0
    pedataFraz = (dy-yLanding)
    newPedata = pedataFraz/(totGradiniIntero)
    larghezzaStair = dx/2.0
    
    #construct landing as a cuboid with the size calculate before
    landing = CUBOID([larghezzaStair*2,yLanding,newAlzata])
    
    #define the diagonale for fill the stairs
    diagonal=[]
    dist=[-(dx/2),dx/2]
    distDiagonal=QUOTE(dist)
    diagonal.append([0,newPedata])
    diagonal.append([newAlzata,newPedata])
    diagonal.append([newAlzata,2*newPedata])
    d=MKPOL([diagonal,[[1,2,3]],None])
    d=PROD([distDiagonal,d])
    d=STRUCT([R([2,3])(PI/2),d])
    d=STRUCT([R([1,2])(PI),d])
    d=STRUCT([T([1])(dx*3/2),d])
    
    #contruct first stair landing
    #i create every each single stair until the total stairs's number. 
    #stair is create as a cuboid with the size calculate before
    stairsLanding=[]
    for i in range (1,int(totGradiniIntero/2)):
        stairsLanding.append(CUBOID([larghezzaStair,newPedata,newAlzata]))
        stairsLanding.append(T([2,3])([newPedata,newAlzata]))
        d=STRUCT([d,T([2,3])([newPedata,newAlzata]),d])
    stairsLanding.append(landing)    
    d=STRUCT([T(1)(-dx/2),d])
    
    #transform diagonal, ramp and their union in a STRUCT
    structD=STRUCT([d])
    structSL=STRUCT(stairsLanding)
    totalRamp=STRUCT([structD,structSL])
    
    #rotate and traslate the second ramp for create the second effective ramp
    ramp2Rotation=STRUCT([R([1,2])(PI),totalRamp])
    totalRamp2 = STRUCT([T([1,2,3])([dx,(newPedata*((totGradiniIntero)/2)-newPedata),(newAlzata*((totGradiniIntero)/2))]),ramp2Rotation])
    
    #assemble the two ramps before the return
    finalTotalRamps=STRUCT([totalRamp,totalRamp2])
    
    return finalTotalRamps
    

Launch function

In [30]:
ramps=ggpl_stair_landing(1.94,5,3.2)
VIEW(ramps)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000001AD80AE0> >

<b>Intermediate steps</b>

<b>totalRamp: just one ramp</b>
![alt text](firstStairLanding.jpg)

<b>ramp2Rotation: 2 ramps without any traslation of the second's</b>
![alt text](noTraslation.jpg)
![alt text](below.jpg)

<h2>Final result with different sizes!</h2>

<b>finalTotalRamps: 2 complete ramps. Sizes: dx=1.94, dy=5,dz=3.2</b>
![alt text](final.jpg)

<b>finalTotalRamps: 2 complete ramps. Sizes: dx=0.73, dy=6,dz=5.2</b>
![alt text](final2.jpg)

<b>finalTotalRamps: 2 complete ramps. Sizes: dx=1.5, dy=4,dz=2</b>
![alt text](final3.jpg)